Giải captcha

In [15]:
from capmonstercloudclient import CapMonsterClient, ClientOptions
from capmonstercloudclient.requests import RecaptchaV2ProxylessRequest

API_KEY = "b238f538e55b7deb0da93267f61d8763"
WEBSITE_URL = 'https://www.shoutout.global/login?id=22wbe'
WEBSITE_KEY = '6LfvfrEUAAAAAPg5Dt1q3UsmCwD_Z5oELX4s95eB'
async def solve_captcha():
    client_options = ClientOptions(api_key=API_KEY)
    cap_monster_client = CapMonsterClient(options=client_options)
    recaptcha2request = RecaptchaV2ProxylessRequest(
        websiteUrl=WEBSITE_URL,
        websiteKey=WEBSITE_KEY
    )
    return await cap_monster_client.solve_captcha(recaptcha2request)

# responses = await solve_captcha()
# print(responses["gRecaptchaResponse"])


Login and get Id


In [16]:
import re
import aiohttp

def extract_id_from_html(html_content):
    match = re.search(r'/userdashboard\?id=([a-zA-Z0-9]+)', html_content)

    if match:
        id_value = match.group(1)
        return id_value
    else:
        return None

async def LoginAndGetIdAsync(login_url, payload, headers):
    async with aiohttp.ClientSession() as session:
        async with session.post(login_url, data=payload, headers=headers, allow_redirects=False) as response:
            if response.status == 200:
                html_content = await response.text()
                id = extract_id_from_html(html_content)
                if id:
                    return id
                else:
                    print("Không tìm thấy id.")
            else:
                print(f"Error {response.status}: {await response.text()}")
            return None


In [17]:

url = 'https://www.shoutout.global/checklogin'

res = await solve_captcha()
payload = {
    "email": "hunghi1090811@gmail.com",
    "password": "c0kL2Q23iGpC",
    "g-recaptcha-response": res["gRecaptchaResponse"],
    "encryptedID": "22wbe"
}
headers = {
    'Origin': 'https://www.shoutout.global'
}
    
id = await LoginAndGetIdAsync(url, payload, headers)
print(f"id = {id}")


id = be511dcfdd614aebc106fda72fb332d0e5c37645ec6cd7548893656ff18e33e34c0f0b6c000f1e479b9ced12089d16d8079c3d94eaf1731c1fe7d943bdcf2237ab5922f3ebcfb20f794505dd5dc3eb113622a388e29b1fa086b9de99913dce3f35003366bf6c7bff86


GET data

In [18]:
import aiohttp
from bs4 import BeautifulSoup

async def fetch_data(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:            
            if response.status == 200:
                html_content = await response.text()
                soup = BeautifulSoup(html_content, 'html.parser')
                
                total_revenue_element = soup.find(id='totalRevenueTxt')
                return {
                    "salesCommissionTxt": soup.select('.card .card-body h1.card-title')[0].get_text(strip=True),
                    "leadTxt": soup.select('.card .card-body h1.card-title')[1].get_text(strip=True),
                    "totalRevenueTxt": total_revenue_element.get_text(strip=True) if total_revenue_element is not None else None,
                    "totalCommissionTxt": soup.select('.card .card-body .col-12 h2')[1].get_text(strip=True),
                    "pendingCommissionTxt": soup.select('.card .card-body .col-12 h2')[2].get_text(strip=True)
                }
            else:
                print(f"Error: {response.status}")
                return None


Crawl data function

In [19]:
from urllib.parse import urlparse, parse_qs

async def crawl_data(args):
    url, email, password = args  
    encryptedID = parse_qs(urlparse(url).query)
    res = await solve_captcha()
    payload = {
        "email": email,
        "password": password,
        "g-recaptcha-response": res["gRecaptchaResponse"],
        "encryptedID": encryptedID["id"][0]
    }
    headers = {
        'Origin': 'https://www.shoutout.global'
    }

    # print(payload)
        
    id = await LoginAndGetIdAsync("https://www.shoutout.global/checklogin", payload, headers)
    if id:
        data = await fetch_data(f"https://www.shoutout.global/userdashboard?id={id}")
        return data
    else:
        print("Failed to obtain ID.")
        return None


Main


In [20]:
import pypeln as pl

data = [
    ("https://www.shoutout.global/login?id=22wbe", "natashacook371sdas@gmail.com", "Qxwg0CN09v"), 
    ("https://www.shoutout.global/login?id=obbi7", "teamasmads@gmail.com", "E9vQRQmPG!a.7m6")
]
stage = await pl.task.map(crawl_data, data, workers=100)
print(stage)


[{'salesCommissionTxt': '1%', 'leadTxt': '0', 'totalRevenueTxt': '$0.00', 'totalCommissionTxt': '$0.00', 'pendingCommissionTxt': '$0.00'}, {'salesCommissionTxt': '30%', 'leadTxt': 'up to5%', 'totalRevenueTxt': '$0.00', 'totalCommissionTxt': '$0.00', 'pendingCommissionTxt': '$0.00'}]
